In [ ]:
 # This cell to remove every varialbe 
import sys   
sys.modules[__name__].__dict__.clear()
print(dir())

# Modules

In [2]:
from datetime import date
import random
import string

# Parameters

In [3]:
List_Unit = ["W2", "W3", "M2", "M4", "MX"]
List_Category = ["11_16", "12_18", "J", "S"]

# Athletes
Each athlete is characterised by a first and last name, a gender, a date of birth and a nationality

In [4]:
class Athlete:
    def __init__(self, fi, la, gen, bir ,natio):
        self.__first  = str(fi).title()    # Double leading underscore: interpreter replaces this name with '_Athlete__first'
        self.__last = str(la).title()
        self.gender = gen
        self.__birth = bir            
        self.__age = self.agef() 
        self.nationality = str(natio).title()
        self.speci = self.__class__.__name__.lower()
    
    def agef(self):
        today = date.today()
        assert self.__birth < today,f'date of birth should be less than today {today}. '
        age = today.year - self.__birth.year - ((today.month, today.day) < (self.__birth.month, self.__birth.day))
        return age  
    
    def __str__(self):
        return f'Name : {self.first}, Surname : {self.last}, \nBirthday : {self.__birth} \nGender : {self.gender} \nNationality: {self.nationality}'
    
    #Athlete comparison based on the attribute .age 
    # Note that just a trivial-quick alternative to direct comparison like  (obj1.age < obj2.age)
    def __lt__(self,other):   # less than
        if isinstance(other, Athlete):
            return self.age<other.age
        return NotImplemented
        
    #Make objects hashable 
    # The hash codes are not allowed to change for an object, so, 
    # to prevent attributes from external accidental mutation, try to make them read-only. Do so: using @property.
    # It is a kind of mask, which however can be violated.
    @property
    def birth(self):
        return self.__birth
    @property
    def age(self):
        return self.__age
    @property
    def first(self):
        return self.__first
    @property
    def last(self):
        return self.__last                
        
    def __eq__(self, other):
        if isinstance(other, Athlete):
            return self.age == other.age  
        return NotImplemented

    def __hash__(self): 
        return hash((self.age,self.first,self.last))      # better than: "return hash(self.age)" Reason: See test below  "st = {k_copy,k_similar,k,k2}"  

# Team

There are five possible teams compositions:

- Women's pair (W2): a team composed of two female athletes
- Women's group (W3): a team composed of three female athletes
- Men's pair (M2): a team composed of two male athletes
- Men's group (M4): a team composed of four male athletes
- Mixed pair (MX): a team composed of a male and a female athletes


Each team competes in a single specific category:

- Age-group 11-16 (11-16): all the team members must be at least 11 years old and at most 16
- Age-group 12-18 (12-18): all the team members must be at least 12 years old and at most 18
- Junior (J): all the team members must be at least 13 years old and at most 19
- Senior (S): all the team members must be at least 15 years old, without an upper bound

In [5]:
class Team:

   def __init__(self, CAT = None, UNI = None, *args):
      self.li_a = []            # list of athletes in Team
      self.number = 0           # total number of athletes
      self.category = str(CAT)
      self.unit = str(UNI)
      self.id = ''.join(random.choice(string.ascii_letters) for i in range(5))  # id code
      self.name =   self.category + " " + self.unit + " " + self.id             # Team name    

      # Built a Team
      for i in args:
        self.li_a.append(i)
        self.number = self.number + 1


   def __str__(self) :
        if self.name[:5] == "None":
           return f'The data entered does not make up a valid Team'
        else:
           return f'Team : {self.name} \nTot athletes: {self.number}'


   #Building Iterator  
   def __iter__(self):       
         self.x = 0                       # initialization
         return self                   

   def __next__(self): 
         if self.x < len(self.li_a):
             result = self.li_a[self.x]
             self.x  = self.x + 1
             return result                # return the next item in the sequence
         else:
             raise StopIteration          #  when it reachs the end
   

   @classmethod                           # generate multiple Team-objects within the same class
   def _multiple(cls, CAT, UNI, l):
        return [Team(CAT, UNI) for i in range(l)]

# Rankings


Rankings are split by category and team composition, e.g., a Junior MX team competes against only other Junior MX teams. Each team in the competition is assigned a unique identifier.


Given the previous information, there could be up to 20 different rankings to be considered.

In [6]:
class Rankings:   
   
   def __init__(self, CAT = None, UNI = None, *args):
      self.li_t = []        # list of  Team
      self.num_t = 0        # total number of Team in ranking
      self.category = str(CAT)
      self.unit = str(UNI)
      self.name_t =   self.category + " " + self.unit + " " +  str(date.today().year) # Ranking name 

       # Built
      for i in args:
        self.li_t.append(i)
        self.num_t = self.num_t + 1
        i.scoring = 0       #  starting score of a certain team. Note : once team are in the ranking they have a scoring attribute

   def __str__(self):
         return f'\\ Rankings : {self.name_t} \nTot Team: {self.num_t}'

   def __len__(self):
         return self.num_t

   #Building Iterator  
   def __iter__(self):       
         self.x = 0                       # initialization
         return self                   

   def __next__(self): 
         if self.x < len(self.li_t):
             result = self.li_t[self.x]
             self.x  = self.x + 1
             return result                # return the next item in the sequence
         else:
             raise StopIteration          #  when it reachs the end


# Exercise  -- Approach Multiple Inheritance


There are three types of exercises:

- Balance (B): an exercise with a focus on strength, poise and flexibility
- Dynamic (D): an exercise with a focus on throws, catches and tumbling elements
- Combined (C): a combination of balance and dynamic elements

Balance and dynamic exercises are performed in qualifiers whereas combined exercises are performed only during finals.



In [7]:
# Approach Multiple inheritance

class Exercise:
   def __init__(self, NE, *args,**kwargs):    
     self.name_ex = NE                       # exercise name
     super().__init__(*args, **kwargs)  


class Balance(Exercise):
   def __init__(self, NE, stre = 0.1, po = 0.2, fle = 0.3, *args,**kwargs):     # float 0.1 , 0.2., 0.3  I figured they are the minimum difficulty levels for the respective feature
     self.strength = float(stre)   # level of difficulty a priori for strength :  a value between 0.00 and 3.00    
     self.poise =  float(po)       # level of difficulty a priori for poise
     self.flexibility =float(fle)  # level of difficulty a priori for flexibility
     self.final = False            # whether or not it runs in the final
     super().__init__(NE, *args,**kwargs)

   def __str__(self):
        return f'Exercise name : {self.name_ex} \nType of exercise : {self.__class__.__name__}'


class Dynamic(Exercise):
   def __init__(self, NE, thro = 0.11 , cat = 0.22, tum = 0.33, *args,**kwargs):
     self.throws = float(thro)           # level of difficulty a priori for throws :  a value between 0.00 and 3.00    
     self.catches =  float(cat)          # level of difficulty a priori for catches
     self.tumbling_elements =float(tum)  # level of difficulty a priori for tumbling elements
     self.final = False            # whether or not it runs in the final
     super().__init__(NE, *args,**kwargs)

   def __str__(self):
        return f'Exercise name : {self.name_ex} \nType of exercise : {self.__class__.__name__}'
     
class Combined(Dynamic,Balance): 
      def __init__(self, NE, stre = 0 , po = 0  , fle = 0  , thro = 0 , cat = 0 , tum = 0 , *args,**kwargs ):
         super().__init__(NE, thro , cat , tum, stre, po , fle, *args,**kwargs)
         self.final = True
    

#Test 1  Multiple inheritance
# class test1(Dynamic): 
#       def __init__(self, NE, thro , cat , tum  , *args, **kwargs ):
#          super().__init__(NE, thro , cat , tum, *args,**kwargs)
#          self.final = True


# Competition setup

The competition is organised in a qualifier round and a final round. All the teams partake in the qualifiers, but only a few are allowed to compete in the finals. For the sake of simplicity, assume that only the first top-8 teams for each category/composition are allowed to move on.

In [8]:
class Match:
   def __init__(self, t_A=None, t_B=None, s=(0,0)):
     self.team_A = t_A
     self.team_B = t_B
     self.score = s

   def __str__(self):
      return(f'{self.team_A.id} {self.team_B.id} {self.score}')

   def result(self):
      self.score = (random.randint(0, 10),random.randint(0, 10))
      if self.score[0] > self.score[1]:
         return self.team_A
      elif self.score[0] < self.score[1]:
         return self.team_B
      else:
         return (random.choice([self.team_A,self.team_B]))  


def Qualifiers(R):                               #Practically:   in : tthe top 8 Team in rankings ; out: one Team winner
     final1 = Match( (Match(R.li_t[0],R.li_t[7])).result(), (Match(R.li_t[1],R.li_t[6]).result()) ).result()
     final2 = Match( (Match(R.li_t[2],R.li_t[5])).result(), (Match(R.li_t[3],R.li_t[4]).result()) ).result()

     winner = Match(final1, final2).result()    # This is the final match

     return winner

# Code testing

In [9]:

print("***Testing class codes Athlete, Team, Rankings\n")
k = Athlete("Aldo", "Ielpo", "M", date(2011, 3, 10) ,"ITA")
print(k)
print(k.__dict__)
print(dir(k))
print(isinstance(k, Athlete))
print("AGE : " , k.age)       #Read Only Properties
print("BIRTH : ", k.birth)    #Read Only Properties
print("BIRTH : ", k._Athlete__birth)   # Can change with this


print("**Objects comparable")
k2 = Athlete("Jose", "Morata", "M", date(2010, 1, 5) ,"SPA")
k_copy = Athlete("Aldo", "Ielpo", "M", date(2011, 3, 10) ,"ITA")
k_similar = Athlete("Pino", "Renzulli", "M", date(2011, 3, 10) ,"ITA")
print(k == k_copy)
print(k == k_similar)
print(k == k2)
print(k < k2)

print("**Objects hashable")
print(hash(k_copy))
print(hash(k))
print(hash(k_similar))
print(hash(k2))
st = {k_copy,k_similar,k,k2}
print("Set : ", st)


print("\n")

print("***Testing class codes Team\n")
u = Team(List_Unit[2],List_Category[0],k,k2)

[print(i) for i in u]
# Practical alternative to use of iterator
# for i in u.li_a:
#   print(i)

print("\n")
print(u)

print("\n")
print("***Testing class codes Rankings\n")

l_t = []
for i in range(7):
  l_t.append(Team(List_Unit[2],List_Category[0]))
l_t.append(u)

kl = Team._multiple(List_Unit[2],List_Category[0],2)  
print(kl[0])
l_t = l_t + kl

rk1 = Rankings(List_Unit[2],List_Category[0],*l_t)

print("\n",rk1)
print(len(rk1))
for z in rk1:
  print(z)
  print("Current scoring : ", z.scoring,"\n")      



***Testing class codes Athlete, Team, Rankings

Name : Aldo, Surname : Ielpo, 
Birthday : 2011-03-10 
Gender : M 
Nationality: Ita
{'_Athlete__first': 'Aldo', '_Athlete__last': 'Ielpo', 'gender': 'M', '_Athlete__birth': datetime.date(2011, 3, 10), '_Athlete__age': 12, 'nationality': 'Ita', 'speci': 'athlete'}
['_Athlete__age', '_Athlete__birth', '_Athlete__first', '_Athlete__last', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'age', 'agef', 'birth', 'first', 'gender', 'last', 'nationality', 'speci']
True
AGE :  12
BIRTH :  2011-03-10
BIRTH :  2011-03-10
**Objects comparable
True
True
False
True
**Objects hashable
3573423092939226284
3573423092939226284
-192299527290732160
-1009371881009322356

In [10]:
print("Testing class Match\n")
# rt = Match(rk1.li_t[0],rk1.li_t[7])
# print(rt.result().id)
# print(rt.score)
fg = Match(rk1.li_t[0],rk1.li_t[7]).result().id
print(fg)

Testing class Match

bJqjb


In [11]:
print("Testing class Qualifiers\n")
wi = Qualifiers(rk1)
print(wi,"\n")
print("The Winner is :",wi.id)

Testing class Qualifiers

Team : M2 11_16 mQZks 
Tot athletes: 0 

The Winner is : mQZks


In [12]:
print("Testing class codes Exercise - Approach Multiple inheritance \n")

ex1 = Dynamic("Pike", 2.7)
print(ex1)
print(ex1.throws)
print(ex1.catches)
print(ex1.tumbling_elements)

print("   ")


ex2 = Combined("Butterfly",stre=1.1, fle=1.8, tum=2.2, po=1.5)
print(Combined.__mro__)
print(ex2)
print(dir(ex2))
print("")
print("In final : ", ex2.final)
print("Dynamic features")
print(ex2.throws)
print(ex2.catches)
print(ex2.tumbling_elements)
print("Balance features")
print(ex2.strength)
print(ex2.poise)
print(ex2.flexibility)



Testing class codes Exercise - Approach Multiple inheritance 

Exercise name : Pike 
Type of exercise : Dynamic
2.7
0.22
0.33
   
(<class '__main__.Combined'>, <class '__main__.Dynamic'>, <class '__main__.Balance'>, <class '__main__.Exercise'>, <class 'object'>)
Exercise name : Butterfly 
Type of exercise : Combined
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'catches', 'final', 'flexibility', 'name_ex', 'poise', 'strength', 'throws', 'tumbling_elements']

In final :  True
Dynamic features
0.0
0.0
2.2
Balance features
1.1
1.5
1.8
